# Assigment - Weakly interacting and confined bosons at low density

### _Authors:_ Clàudia Garcia and Adrià Rojo

L'equació de Gross-Pitaevskii (GP) ens permet descriure els experiments de condensats de Bose-Einstein dins del marc tèoric. 

En aquest exemple, considerem N àtoms de $^{87}$ Rb, on tots els àtoms estàn confinats per un camp magnètic, i tots els efectes d'aquest es descriuen bé per un potencial harmònic. També considerem que $\Psi(\vec{r})$ està normalitzat a la unitat. L'equació GP és la següent:

$$
\left[-\frac{1}{2} \nabla^2+\frac{1}{2} r_1^2+4 \pi \bar{a}_s N\left|\bar{\Psi}\left(\vec{r}_1\right)\right|^2\right] \bar{\Psi}\left(\vec{r}_1\right)=\bar{\mu} \bar{\Psi}\left(\vec{r}_1\right),
$$

on $\mu$ és el potencial químic, $N$ el nombre de partícules i $\bar{a}_s$ és la longitud de $s$-wave scattering. L'equació està escrita en unitats de oscil·lador harmònic.

Començarem per precomputar valors de distància i variables de la simulació i la funció d'ona inicial i fer tots els `imports`. Per la segona derivada utilitzarem la funció `gradient` de `numpy`. Utilitzarem `tqdm` per iterar el bucle i tenir una estimació del temps d'execució. El codi utilitza de forma extensiva generadors, per evitar l'anidament en un segon bucle `for`. 

In [43]:
import numpy as np
import numpy.typing as npt
from tqdm import tqdm
from IPython.display import display, Math, Markdown, clear_output
import typing as t

def second_derivative(fun: npt.ArrayLike, h: float) -> npt.ArrayLike:
    first = np.gradient(fun, h)
    return np.gradient(first, h)



def initial_values(scattering_length: float, width:float, step:float, atom_numbers:int, alpha:float) -> \
    t.Tuple[float, float, float, float, npt.ArrayLike, npt.ArrayLike, npt.ArrayLike]:
    alpha2 = alpha*alpha
    cvar = 2 * np.power(alpha, 3/2)/ np.power(np.pi, 1/4)
    rs = np.array([i * step for i in range(width)])
    r2s = rs**2
    psi = np.array([cvar*r*np.exp(-0.5*alpha2*r2) for (r, r2) in zip(rs, r2s)])
    interaction = scattering_length*atom_numbers
    density_param = atom_numbers*scattering_length*scattering_length*scattering_length
    return alpha2, cvar, interaction, density_param,rs, r2s, psi


La funció `final_properties` calcularà propietats finals de la simulació (no se podia saber), calculant els seguents valors:

$$ \langle r^2 \rangle = \int^\infty_0{r^2\left| \psi^2(r_1)\right|^2 dr_1} \approx \sum^\infty_{r_i = 0}{r_i^2\left| \psi(r_i)\right|^2}\Delta r \qquad
\langle T \rangle = -\frac{1}{2}\int_0^\infty \psi(r) \frac{d^2\psi}{dr^2} \, dr \approx -\frac{1}{2}\sum_i \psi(r_i) \left(\frac{d^2\psi}{dr^2}\right)_i \Delta r \qquad  \\
\langle V_{\text{harm}} \rangle = \frac{1}{2}\int_0^\infty r^2 |\psi(r)|^2 \, dr \approx \frac{1}{2}\sum_i r_i^2 |\psi(r_i)|^2 \Delta r \qquad
\langle V_{\text{int}} \rangle = \frac{g}{2}\int_0^\infty \frac{|\psi(r)|^4}{r^2} r^2 \, dr \approx \frac{g}{2}\sum_i r_i^2 \left(\frac{|\psi(r_i)|}{r_i}\right)^4 \Delta r \qquad \\
\langle \mu \rangle = \int_0^\infty \mu(r) |\psi(r)|^2 \, dr \approx \sum_i \mu(r_i) |\psi(r_i)|^2 \Delta r \qquad  
\mu(r) = \frac{-\frac{1}{2}\frac{d^2\psi}{dr^2} + \frac{1}{2}r^2\psi(r) + g\frac{|\psi(r)|^2}{r^2}\psi(r)}{\psi(r)} \qquad \\
E_{\text{total}} = \langle T \rangle + \langle V_{\text{harm}} \rangle + \langle V_{\text{int}} \rangle \qquad
n(r) = \begin{cases} 
\left(\frac{\psi(r)}{r}\right)^2 & \text{si } r \neq 0 \\
0 & \text{si } r = 0
\end{cases}
$$

In [ ]:
def final_properties(psi: npt.ArrayLike, local_mu: npt.ArrayLike, step: float, rs: npt.ArrayLike, r2s:npt.ArrayLike, interaction: float) -> \
    t.Dict[str, float]:
    psi_dx2 = second_derivative(psi, step)
    
    # integrals
    local_values = [
        (
            r2*final*final, #acc radius
            final*dx2, # kin energy
            r2*final*final, # armonic
            r2*(final/r if r != 0 else 1)**4 , # interaction energy
            mu*final*final, # average chem potential
            r2/2 + interaction * (final/r if r != 0 else 1)**2, # particle potential
            (final/r if r != 0 else 1)**2  # density
        )
        for r, r2, final, dx2, mu in zip(rs, r2s, psi, psi_dx2, local_mu)
    ]

    local_accumulted_radius, local_kinetic_energy, local_armonic_potential, local_interaction_energy, local_average_chem_potential, particle_potential, density =\
        zip(*local_values)

    squared_radius = sum(local_accumulted_radius)*step
    mean_squared_radius = np.sqrt(squared_radius)
    average_chem_potential = sum(local_average_chem_potential)*step
    kinetic_energy = -sum(local_kinetic_energy)*step/2
    armonic_potential = sum(local_armonic_potential)*step/2
    interaction_energy = sum(local_interaction_energy)*step*interaction/2

    return {
        "r^2": squared_radius,
        "<r^2>": mean_squared_radius,
        "avg mu": average_chem_potential,
        "kin energy": kinetic_energy,
        "armonic pot" : armonic_potential,
        "interac energy": interaction_energy,
        "total energy": kinetic_energy+armonic_potential+interaction_energy
    }

def compute_wavelength(rs: npt.ArrayLike, r2s: npt.ArrayLike, psi: npt.ArrayLike, iterations: int, step: float, interaction: float, time_step: float, label: str=None) -> \
        t.Tuple[npt.ArrayLike, npt.ArrayLike]:
    psi_curr = psi
    for i in tqdm(range(iterations), desc=label):
        psi_dx2 = second_derivative(psi_curr, step)
        # calculo de energia y mu local
        local_energy_mu = [(-(curr*dx2+r2*curr**2+interaction*r2*(curr/r if r != 0 else 1)**4)/2 , 
                            (-(dx2/curr if curr != 0 else 1) + r2 + interaction*(curr/r if r != 0 else 1)**2)/2 )
                        for r, r2, curr, dx2 in zip(rs, r2s, psi_curr, psi_dx2)]
        # deshacer la lista de tuplas (energía y mu de cada distancia) 
        # en una tupla de listas (lista de todas las energías y lista de todas las mus)
        local_energy, local_mu = zip(*local_energy_mu)
        energy = sum(local_energy)*step
        # evolución de la función de onda
        psi_next = [curr*(1 - time_step*mu) for curr, mu in zip(psi_curr, local_mu)]
        normalization_const = np.sqrt(sum(next*next for next in psi_next)*step)
        psi_curr = np.array(psi_next)/normalization_const
    
    return (psi_curr, local_mu)

### a0) Terme d'interacció nul.

Si considerem el terme d'interacció nul (en el nostre programa la variable `interaction`), l'equació GP queda com: 

$$
\left[-\frac{1}{2} \nabla^2+\frac{1}{2} r_1^2\right] \bar{\Psi}\left(\vec{r}_1\right)=\bar{\mu} \bar{\Psi}\left(\vec{r}_1\right),
$$

Que com veiem
$
H_{o.h} = -\frac{1}{2} \nabla^2+\frac{1}{2} r_1^2,
$
 coincideix amb l'expressió de l'hamiltonià de l'oscil·lador harmònic en tres dimensions, per tant, tenim que: 
$H_{o.h} \bar{\Psi}\left(\vec{r}_1\right) = \varepsilon \bar{\Psi}\left(\vec{r}_1\right)$ i comparant ambdós equacions obtenim que $\bar{\mu} = \varepsilon$, és a dir, el potencial químic coincideix amb l'energia per partícula ($ \varepsilon = E/N$).

In [24]:
scattering_length, width,  step, atom_numbers, time_step, alpha, iterations = \
          0.00433,  400,  0.015,        10000,    0.0001,   0.8,      70000

alpha2, cvar, interaction, density_param, rs, r2s, psi = \
    initial_values(scattering_length, width, step, atom_numbers, alpha)
interaction = 0 # sobreescribimos el valor

final, local_mu = compute_wavelength(rs, r2s, psi, iterations,step, interaction,time_step)

properties = final_properties(final, local_mu, step, rs, r2s, interaction)

display(Math("E = {total energy:.5f}, \\mu = {avg mu:.5f}".format(**properties)))
display(Math("\\varepsilon_{{kin}} = {kin energy:.5f}, \\varepsilon_{{o.h}} = {armonic pot:.5f}, \\varepsilon_{{int}} = {interac energy:.5f}".format(**properties)))

100%|██████████| 70000/70000 [00:31<00:00, 2215.28it/s]


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Podem observar que tant l'energia cinética com la de l'oscil·lador harmònic es reparteixen de forma igual l'energia, complint d'aquesta forma el teorema del virial per aquests casos. Evidentment, el terme d'energia d'interacció és igual a zero.

### a) Resolució de l'equació GP per a diferents $N$

Utilizant $\bar{a}_s = 0.00433$ resolem l'equació GP per diferents nombres de partícules ($N$) i obtenim els següents resultats per $\mu$ i les següents energies per partícules: total, cinètica, de l'oscil·lador harmònic i d'interacció.



In [ ]:
markdown_table = "| $N$   | $\\bar{\\mu}$ | $\\varepsilon$ | $\\varepsilon_{kin}$ | $\\varepsilon_{o.h}$ | $\\varepsilon_{int}$ |\n" +\
                 "|:-----:|:-----------:|:---:|:---------:|:---------:|:---------:|\n"

display(Markdown(markdown_table))

normal_props = {}

for exponent in range(2, 8):
    particles = int(10**exponent)
    scattering_length, width,   step, atom_numbers, time_step, alpha, iterations = \
              0.00433,   400,  0.015,    particles,    0.0001,   0.8,      40000

    alpha2, cvar, interaction, density_param, rs, r2s, psi = \
        initial_values(scattering_length, width, step, atom_numbers, alpha)
    final, local_mu = compute_wavelength(rs, r2s, psi, iterations,step, interaction,time_step, label="N = 10^{0:02d}".format(exponent))

    properties = final_properties(final, local_mu, step, rs, r2s, interaction)
    normal_props[exponent] = properties
    markdown_table += "|$10^{{ {exponent} }}$|{avg mu:.5f}|{total energy:.5f}|{kin energy:.5f}|{armonic pot:.5f}|{interac energy:.5f}|\n".format(exponent=exponent, **properties)
    clear_output(True)
    display(Markdown(markdown_table))

clear_output(True)
display(Markdown(markdown_table))


| $N$   | $\bar{\mu}$ | $\varepsilon$ | $\varepsilon_{kin}$ | $\varepsilon_{o.h}$ | $\varepsilon_{int}$ |
|:-----:|:-----------:|:---:|:---------:|:---------:|:---------:|
|$10^{ 2 }$|1.65589|1.65589|0.69589|0.80908|0.15092|
|$10^{ 3 }$|2.48689|2.48689|0.50978|1.13860|0.83851|
|$10^{ 4 }$|5.29692|5.29692|0.29058|2.31160|2.69474|
|$10^{ 5 }$|12.81550|12.81550|0.15246|5.51201|7.15103|
|$10^{ 6 }$|40.98343|40.98343|0.02722|10.03987|30.91634|
|$10^{ 7 }$|6414816481.48159|6414816481.48160|1666.66676|0.00011|6414814814.81472|


Observem que l'energia d'interacció augmenta amb la quantitat de partícules i en canvi l'energía cinética disminueix, estant més d'acord cada cop amb l'aproximació de Thomas-Fermi. Veiem que a partir de $N=10^7$ hi ha un error d'_overflow_.

### b) Aproximació de Thomas-Fermi.
 En el marc d'aquesta aproximació el terme cinètic es pot considerar nul i l'equació GP queda com: 

$$
\left[\frac{1}{2} r_1^2+4 \pi \bar{a}_s N\left|\bar{\Psi}\left(\vec{r}_1\right)\right|^2\right] \bar{\Psi}\left(\vec{r}_1\right)=\bar{\mu} \bar{\Psi}\left(\vec{r}_1\right)
$$

Imponent que el límit de la simulació és $r_{TF} = (15 g)^{1/5}$, i que la funció del potencial químic local tingui un tall abrupte a $r_{TF}$.

In [55]:
def initial_values_TF(scattering_length: float, width: int, step: float, atom_numbers: int) ->\
      t.Tuple[npt.ArrayLike, npt.ArrayLike, npt.ArrayLike, float, float]:
    g = scattering_length * atom_numbers  # Constante de interacción efectiva
    mu_TF = (15 * g )**(2/5)  # μ_TF en 3D
    R_TF = np.sqrt(2 * mu_TF)  # Radio de Thomas-Fermi
    
    rs = np.array([i * step for i in range(width)])
    r2s = rs**2
    psi_TF = np.array([np.sqrt(max((mu_TF - 0.5 * r2) / g, 0)) for r2 in r2s])
    
    return g, rs, r2s, psi_TF, mu_TF, R_TF

def compute_mu_local_TF(rs: npt.ArrayLike, mu_TF: float, R_TF: float) ->\
    npt.ArrayLike:
    return np.array([mu_TF if r <= R_TF else 0 for r in rs])

def final_properties_TF(
    rs: npt.ArrayLike, 
    r2s: npt.ArrayLike, 
    step: float, 
    interaction: float, 
) -> t.Dict[str, float]:

    mu_TF = (15 * interaction)**(2/5)
    R_TF = np.sqrt(2 * mu_TF)
    
    density = np.array([
        (mu_TF - 0.5 * r2) / interaction if (mu_TF - 0.5 * r2) > 0 and r != 0 else 0 
        for r, r2 in zip(rs, r2s)
    ])
    
    local_values = [
        (
            r2 * dens,
            0.0,
            r2 * dens,
            r2 * dens**2,
            mu_TF * dens,
            0.5 * r2 + interaction * dens,
        )
        for r2, dens in zip(r2s, density)
    ]
    
    # Desempaquetar valores locales
    (local_radius, _, local_harmonic, local_interaction, 
     local_mu, _) = zip(*local_values)
    
    # Integrales (suma * paso)
    squared_radius = sum(local_radius) * step
    harmonic_pot = 0.5 * sum(local_harmonic) * step
    interaction_energy = 0.5 * interaction * sum(local_interaction) * step
    avg_mu = sum(local_mu) * step
    
    return {
        "r^2": squared_radius,
        "<r^2>": np.sqrt(squared_radius),
        "avg mu": avg_mu,
        "kin energy": 0,  # Cero en TF
        "armonic pot": harmonic_pot,
        "interac energy": interaction_energy,
        "total energy": harmonic_pot + interaction_energy,
        "R_TF": R_TF,
        "mu_TF": mu_TF
    }

In [53]:
markdown_table = "| $N$   | $\\bar{\\mu}$ | $\\varepsilon$ | $\\varepsilon_{kin}$ | $\\varepsilon_{o.h}$ | $\\varepsilon_{int}$ |\n" +\
                 "|:-----:|:-----------:|:---:|:---------:|:---------:|:---------:|\n"
TF_props = {}

display(Markdown(markdown_table))
for exponent in range(2, 7):
    particles = int(10**exponent)
    scattering_length, width,   step, atom_numbers, time_step, alpha, iterations = \
              0.00433,   400,  0.015,    particles,    0.0001,   0.8,      40000
    
    interaction, rs, r2s, psi_TF, mu_TF, R_TF = \
        initial_values_TF(scattering_length, width, step, atom_numbers)
    local_mu = compute_mu_local_TF(rs, mu_TF, R_TF)

    properties = final_properties_TF(rs,r2s,step,interaction)
    markdown_table += "|$10^{{ {exponent} }}$|{avg mu:.5f}|{total energy:.5f}|{kin energy:.5f}|{armonic pot:.5f}|{interac energy:.5f}|\n".format(exponent=exponent, **properties)
    TF_props[exponent] = properties
    clear_output(True)
    display(Markdown(markdown_table))

clear_output(True)
display(Markdown(markdown_table))


| $N$   | $\bar{\mu}$ | $\varepsilon$ | $\varepsilon_{kin}$ | $\varepsilon_{o.h}$ | $\varepsilon_{int}$ |
|:-----:|:-----------:|:---:|:---------:|:---------:|:---------:|
|$10^{ 2 }$|14.06464|6.24447|0.00000|2.82831|3.41616|
|$10^{ 3 }$|14.09332|11.40945|0.00000|2.82843|8.58101|
|$10^{ 4 }$|14.11134|24.38297|0.00000|2.82844|21.55453|
|$10^{ 5 }$|12.73629|46.49239|0.00000|1.88238|44.61001|
|$10^{ 6 }$|9.08980|45.38243|0.00000|0.60774|44.77469|


### c) Plots de la de densitat $\rho(r_1)$ per $N = 1000$ i $N = 100000$. 

Compararem els resultats per l'equació GP i per aquesta en l'aproximació de TF. 

La densitat $\rho(r_1)$ està normalitzada com: $\int dr_1 r_1^2 \rho(r_1)$

En el cas de l'aproximació TF tenim la següent expressió per la densitat: 
$$
\rho(r_1) = \frac{1}{4 \pi N \bar{a}_s} \cdot \left( \bar{\mu} - \frac{1}{2}r_1^2 \right)
$$

### d) Teorema del Virial per diferents valors de $N$

Comprovarem que les solucions obtingudes de l'equació GP compleix el Teorema del Virial. 

El teorema del Virial en el marc de l'equació GP és de la forma: 
$$
2\varepsilon_{kin} - 2\varepsilon_{o.h} + 3\varepsilon_{int} = 0
$$

| $N$   | $2\varepsilon_{kin} - 2\varepsilon_{o.h} + 3\varepsilon_{int}$ | $- 2\varepsilon_{o.h}^{TF} + 3\varepsilon_{int}^{TF}$ | 
|:-----:|:-----------:|:-------------:|
|100    |             |     |
|1000   |
|10000  |
|100000 |
|1000000|

comment behaviour...